In [1]:
import psycopg2
from config import config

In [2]:
def connect():
    '''Connect to the PostGresQL Database Server'''
    conn = None
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        print(db_version)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__=='__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.5.25 on x86_64-apple-darwin21.1.0, compiled by Apple clang version 13.0.0 (clang-1300.0.29.3), 64-bit',)
Database connection closed.


In [17]:
def get_info(query):
    '''this function queries the database and find the information specified in the parameter'''
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        return results
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [18]:
# what is the email address for the customer with the name Nancy Thomas?
get_info("SELECT email FROM customer WHERE first_name = 'Nancy' AND last_name = 'Thomas'")

[('nancy.thomas@sakilacustomer.org',)]

In [19]:
# give a customer the description of the movie "Outlaw Hanky"
get_info("SELECT description FROM film WHERE title = 'Outlaw Hanky';")

[('A Thoughtful Story of a Astronaut And a Composer who must Conquer a Dog in The Sahara Desert',)]

In [20]:
# get the customer number for the customer who lives at 259 Ipoh Drive
get_info("SELECT phone FROM address WHERE address = '259 Ipoh Drive';")

[('419009857119',)]

In [21]:
# get more information about the customer who lives at 259 Ipoh Drive
get_info("""SELECT 
            c.first_name, c.last_name, c.email, a.address, a.phone 
        FROM customer c 
            LEFT JOIN 
                    address a 
                ON c.address_id = a.address_id 
            WHERE a.address = '259 Ipoh Drive';""")

[('Jesus',
  'Mccartney',
  'jesus.mccartney@sakilacustomer.org',
  '259 Ipoh Drive',
  '419009857119')]

In [22]:
# what are the customer ids of the 10 first customers who created a payment?
get_info("SELECT customer_id from payment ORDER BY payment_date ASC LIMIT 10;")

[(416,), (516,), (239,), (592,), (49,), (264,), (46,), (481,), (139,), (595,)]

In [23]:
# what are the titles of the 5 shortest movies?
get_info("SELECT title, length FROM film ORDER BY length ASC LIMIT 5;")

[('Iron Moon', 46),
 ('Kwai Homeward', 46),
 ('Alien Center', 46),
 ('Ridgemont Submarine', 46),
 ('Labyrinth League', 46)]

In [24]:
# how many movies can a customer watch that are less than or equal to 50 minutes long?
get_info("SELECT COUNT(*) FROM film WHERE length <= 50;")

[(37,)]

In [26]:
# how many payment transactions were greater thn $5.00?
get_info("SELECT COUNT(amount) FROM payment WHERE amount > 5;")

[(3618,)]

In [29]:
# how many actors have a first name that starts with the letter 'P'?
get_info("SELECT COUNT(*) FROM actor WHERE first_name ILIKE 'p%';")

[(5,)]

In [30]:
# how many unique districts are our customers from?
get_info("SELECT COUNT(DISTINCT(district)) FROM address;")

[(378,)]

In [33]:
# retrieve the list of names for those distinct districts from the previous query
get_info("""SELECT DISTINCT(district) FROM address;""")

[('Asuncin',),
 ('Yerevan',),
 ('Nagasaki',),
 ('Nonthaburi',),
 ('Oriental',),
 ('Carabobo',),
 ('Bursa',),
 ('Cagayan Valley',),
 ('Mwanza',),
 ('West Bengali',),
 ('Jiangxi',),
 ('Daugavpils',),
 ('Anhalt Sachsen',),
 ('Shandong',),
 ('Anzotegui',),
 ('Lombardia',),
 ('Sinaloa',),
 ('Asir',),
 ('Andhra Pradesh',),
 ('Nampula',),
 ('Khartum',),
 ('Nord-Ouest',),
 ('Scotland',),
 ('Batman',),
 ('Baijeri',),
 ('Basel-Stadt',),
 ('Krasnojarsk',),
 ('Ahal',),
 ('Zanzibar West',),
 ('Veracruz',),
 ('Bretagne',),
 ('Cheju',),
 ('Osaka',),
 ('Botosani',),
 ('Missouri',),
 ('Aden',),
 ('North West',),
 ('St Thomas',),
 ('Adygea',),
 ('Zufar',),
 ('Pernambuco',),
 ('Henan',),
 ('Esfahan',),
 ('Songkhla',),
 ('Sousse',),
 ('Jalisco',),
 ('Baja California',),
 ('Southern Mindanao',),
 ('al-Qadarif',),
 ('Sucre',),
 ('Cear',),
 ('Central Java',),
 ('Pays de la Loire',),
 ('Sumqayit',),
 ('al-Sharqiya',),
 ('Mie',),
 ('Central Serbia',),
 ('Rajasthan',),
 ('Illinois',),
 ('Paran',),
 ('al-Manama'

In [34]:
# how many films are R-rated and have a replacement cost between $5 and $15?
get_info("""SELECT COUNT(*) FROM film WHERE rating = 'R' AND replacement_cost BETWEEN 5 AND 15;""")

[(52,)]

In [35]:
# how many films have the word Truman somewhere in the title?
get_info("""SELECT COUNT(title) FROM film WHERE title ILIKE '%truman%';""")

[(5,)]

In [36]:
# we have two staff members with staff ids 1 and 2. A bonus will be given to the staff member
#  who handled the most transactions. how many payments did each staff member handle and who gets the bonus?
get_info("""
        SELECT staff_id, COUNT(amount) FROM payment GROUP BY staff_id;
""")

[(2, 7304), (1, 7292)]

In [38]:
# corporate hq is conducting a study on the relationship  between replacement cost and a movie rating.
# what is the average replacement cost per rating?
get_info("""
        SELECT rating, ROUND(AVG(replacement_cost), 2) FROM film GROUP BY rating;
""")

[('R', Decimal('20.23')),
 ('PG', Decimal('18.96')),
 ('PG-13', Decimal('20.40')),
 ('NC-17', Decimal('20.14')),
 ('G', Decimal('20.12'))]

In [40]:
# we are running a promotiom to reward 5 customers with coupons.
# what are the customer ids of the 5 customers who have the most payments?
get_info(
    """
        select customer_id, sum(amount) from payment group by customer_id order by sum(amount) desc limit 5;
    """
)

[(148, Decimal('211.55')),
 (526, Decimal('208.58')),
 (178, Decimal('194.61')),
 (137, Decimal('191.62')),
 (144, Decimal('189.60'))]

In [42]:
# we are launching a platinum service for our most loyal customers. we will assidn a platinum membership to
# the customers who have made 40 or more payments. what customer_ids are eligible for platinum status?
get_info(
    """
    SELECT customer_id FROM payment GROUP BY customer_id HAVING COUNT(*) >= 40;
    """)

[(526,), (148,), (144,)]

In [46]:
# what are the customer ids of customers who have spent more than 100 in payment transactions with staff_id 2?
get_info(
    """
    SELECT customer_id, SUM(amount) 
    FROM payment 
    WHERE staff_id = 2 
    GROUP BY customer_id, staff_id 
    HAVING SUM(amount) > 100;
    """
    )

[(187, Decimal('110.81')),
 (526, Decimal('101.78')),
 (211, Decimal('108.77')),
 (148, Decimal('110.78')),
 (522, Decimal('102.80'))]

In [47]:
# return the customer id of customers who have spent at least 110 with the staff member with staff_id 2
get_info("SELECT customer_id from payment WHERE staff_id = 2 GROUP BY customer_id HAVING SUM(amount) > 110;")

[(187,), (148,)]

In [49]:
# how many films begin with the letter 'J'?
get_info("SELECT COUNT(*) from film WHERE title ILIKE 'j%';")

[(20,)]

In [53]:
# what customer has the highest customer id whose name starts with an 'E' and has an address ID lower than 500?
get_info("""
        SELECT MAX(customer_id), first_name, last_name
        FROM customer
        WHERE first_name ILIKE 'e%' 
        GROUP BY customer_id, first_name, last_name
        HAVING address_id < 500
        ORDER BY MAX(customer_id) DESC
        LIMIT 1;
""")

[(434, 'Eddie', 'Tomlin')]